In [511]:
import MetaTrader5 as mt5
import pandas as pd
from datetime import datetime
import time
import requests
import pandas_ta as pta
import numpy as np

In [512]:
def pull_data(symbol, timeframe, _start, _end):
    
    mt5.initialize()

    login = 121391564
    password = 'xxxxxxxx'
    server = 'Exness-MT5Trial7'
    mt5.login(login, password, server)

    raw_df = mt5.copy_rates_range(symbol, timeframe, _start, _end)
    df = pd.DataFrame(raw_df)

    mt5.shutdown()

    df.loc[:,'time'] = pd.to_datetime(df.loc[:,'time'], unit = 's')

    return df

In [513]:
# function to look for trading signals, what trend is now?

def convert_rsi_signal(df, rsi_period = 14, overbought = 70, oversold = 30) :
    if 'close' in df.columns:
        df['rsi'] = pta.rsi(df.close, rsi_period)
        df['signal'] = 'normal'
        df.loc[df['rsi'] < oversold, 'signal'] = 'buy'
        df.loc[df['rsi'] > overbought, 'signal'] = 'sell'
        return df
    else :
        return "Error RSI"

In [594]:
def backtest_rsi(df, rsi_period = 14, overbought = 70, oversold = 30, tp_buy = 50, tp_sell = 50):
    
    t0 = time.time()
    #Genarate signals and details
    df = convert_rsi_signal(df, rsi_period, overbought, oversold)
    df['cross_down'] = (df['rsi'] < tp_buy ) & (df['rsi'].shift(1) > tp_buy)
    df['cross_up'] = (df['rsi'] > tp_sell) & (df['rsi'].shift(1) < tp_sell)
    df['position_price'] = df['open'].shift(-1)
    
    t1 = time.time() - t0
    #Record positions history
    status = 0
    current_order = 'empty'
    df['open_position_price'] = None
    df['profit'] = 0
    df['drawdown'] = 0
    df['cum_profit'] = 0
    cum_profit = 0
    t2 = time.time() - t1 - t0
    
    for ind in df.index:
        #Time series run time
        if df['signal'][ind] == 'buy':
            if status == 0:
                order_price = df['position_price'][ind]
                status = 1
                current_order = 'buy'
        elif df['signal'][ind] == 'sell':
            if status == 0:
                order_price = df['position_price'][ind]
                status = 1
                current_order = 'sell'
        if status == 1:
            if df['cross_up'][ind] or df['cross_down'][ind]:
                profit = df.loc[ind, 'position_price'] - order_price
                if current_order == 'sell':
                    profit *= -1
                df.loc[ind, 'profit'] = profit
                df.loc[ind, 'open_position_price'] = order_price
                cum_profit += profit
                status = 0
                current_order = 'empty'

        df.loc[ind, 'cum_profit'] = cum_profit
        #Drawdown
        if current_order == 'buy':
            df.loc[ind, 'drawdown'] = ((df.loc[ind, 'low'] - order_price)/order_price)*100
        elif current_order == 'sell':
            df.loc[ind, 'drawdown'] = ((order_price - df.loc[ind, 'high'])/order_price)*100
    
    t3 = time.time() - t2 - t1 - t0
    total_profit = df['profit'].sum()
    max_dd = df['drawdown'].min()

    return df, total_profit, max_dd, (t1, t2, t3)


In [595]:
SYMBOL = "XAUUSD"
TIMEFRAME = mt5.TIMEFRAME_H4
PERIOD = 14
OVERBOUGHT = 70
OVERSOLD = 30
TP_BUY = 50
TP_SELL = 50

START_DATE = datetime(2021, 3, 19) #h1 pull after 2021-3-19
END_DATE = datetime(2022, 12, 31)

df = pull_data(SYMBOL, TIMEFRAME, START_DATE, END_DATE)
df, profit, max_dd, t = backtest_rsi(df, PERIOD, OVERBOUGHT, OVERSOLD, TP_BUY, TP_SELL)
profit, max_dd

(41.59499999999957, -6.343520386643328)

In [596]:
df_test = pull_data(SYMBOL, TIMEFRAME, datetime(2023, 1, 1), datetime(2023, 12, 31))
df_test, profit_test, max_dd_test, _ = backtest_rsi(df_test, PERIOD, OVERBOUGHT, OVERSOLD, TP_BUY, TP_SELL)
profit_test, max_dd_test

(-5.462999999999283, -7.873627349788764)

### BUY OPTIMIZATION

In [ ]:
# function to look for trading signals, what trend is now?

def convert_rsi_signal_buy(df, rsi_period = 14, oversold = 30) :
    if 'close' in df.columns:
        df['rsi'] = pta.rsi(df.close, rsi_period)
        df['signal'] = 'normal'
        df.loc[df['rsi'] < oversold, 'signal'] = 'buy'
        return df
    else :
        return "Error RSI"

In [ ]:
#buy order opm
def backtest_rsi_buy(df, rsi_period = 14,oversold = 30, tp_buy = 50):
    
    t0 = time.time()
    #Genarate signals and details
    df = convert_rsi_signal_buy(df, rsi_period, oversold)
    df['cross_up'] = (df['rsi'] > tp_buy ) & (df['rsi'].shift(1) < tp_buy)
    df['position_price'] = df['open'].shift(-1)
    
    t1 = time.time() - t0
    #Record positions history
    status = 0
    current_order = 'empty'
    df['profit'] = 0
    df['drawdown'] = 0

    t2 = time.time() - t1 - t0
    
    for ind in df.index:
        #Time series run time
        if df['signal'][ind] == 'buy':
            if status == 0:
                order_price = df['position_price'][ind]
                status = 1
                current_order = 'buy'

        if status == 1:
            if df['cross_up'][ind]:
                profit = df.loc[ind, 'position_price'] - order_price
                df.loc[ind, 'profit'] = profit
                status = 0
                current_order = 'empty'

        #Drawdown
        if current_order == 'buy':
            df.loc[ind, 'drawdown'] = df.loc[ind, 'low'] - order_price
    
    t3 = time.time() - t2 - t1 - t0
    total_profit = df['profit'].sum()
    max_dd = df['drawdown'].min()

    return df, total_profit, max_dd, (t1, t2, t3)


In [ ]:
START_DATE = datetime(2021, 3, 19) #h1 pull after 2021-3-19
END_DATE = datetime(2022, 12, 31)
df = pull_data(SYMBOL, TIMEFRAME, START_DATE, END_DATE)
df, result, max_dd, t = backtest_rsi_buy(df, 20, 22, 40)
result, max_dd, t

(9488.160000000002,
 -8187.989999999998,
 (0.013965606689453125, 0.0010037422180175781, 0.23272943496704102))

In [ ]:
#for buy
rsi_periods_range = np.arange(10, 22, 2) #10
OVERSOLD_range = np.arange(16,32,2) #8
TP_BUY_range = np.arange(40,64,4) #6

START_DATE = datetime(2021, 3, 19) #h1 pull after 2021-3-19
END_DATE = datetime(2022, 12, 31)

df = pull_data(SYMBOL, TIMEFRAME, START_DATE, END_DATE)
buy_opm = []

for _rsi_period in rsi_periods_range:
    for ovs in OVERSOLD_range:
        for tp_buy in TP_BUY_range:
            df, result, max_dd, t = backtest_rsi_buy(df, _rsi_period, ovs, tp_buy)
            buy_opm.append((_rsi_period, ovs, tp_buy, result, max_dd))
            print(_rsi_period, ovs, tp_buy, result, max_dd)
print(buy_opm)

10 16 40 -4926.770000000013 -11738.239999999998
10 16 44 -4149.830000000033 -11738.239999999998
10 16 48 -3883.8099999999904 -11738.239999999998
10 16 52 -3540.2000000000153 -11738.239999999998
10 16 56 -8624.75000000001 -11738.239999999998
10 16 60 -6750.729999999998 -12311.41
10 18 40 -6593.139999999985 -11738.239999999998
10 18 44 -5116.890000000014 -11738.239999999998
10 18 48 -7850.889999999979 -11738.239999999998
10 18 52 -5477.910000000013 -11738.239999999998
10 18 56 -2463.7300000000178 -11738.239999999998
10 18 60 -1015.0800000000054 -12311.41
10 20 40 -9294.26999999999 -11738.239999999998
10 20 44 -7223.730000000021 -11738.239999999998
10 20 48 -8951.059999999969 -11738.239999999998
10 20 52 -5731.62000000001 -11738.239999999998
10 20 56 -3335.200000000017 -11738.239999999998
10 20 60 -5813.089999999991 -14100.279999999999
10 22 40 -13511.070000000007 -12847.75
10 22 44 -13322.760000000024 -12847.75
10 22 48 -14310.859999999982 -12847.75
10 22 52 -5684.220000000023 -12847.75


In [ ]:
test_buy = pd.DataFrame(buy_opm)
test_buy.columns = 'period', 'ovs', 'tp_buy', 'profit', 'drawdown'
print(test_buy.sort_values('profit', ascending= False).head(10))

     period  ovs  tp_buy   profit  drawdown
258      20   22      40  9488.16  -8187.99
157      16   20      44  8362.18  -8187.99
261      20   22      52  7357.24  -8187.99
103      14   18      44  5824.47  -8187.99
104      14   18      48  5689.76  -8187.99
211      18   22      44  5399.58  -9935.88
156      16   20      40  4866.48  -8187.99
259      20   22      44  4545.76  -8187.99
50       12   16      48  4483.99  -8187.99
260      20   22      48  4189.47  -8187.99
     period  ovs  tp_buy   profit  drawdown
194      18   16      48   142.83  -5574.18
240      20   16      40  1367.09  -5574.18
193      18   16      44  1359.91  -5574.18
192      18   16      40  1342.04  -5574.18
197      18   16      60  1173.32  -5574.18
245      20   16      60  1321.92  -5574.18
244      20   16      56  2469.01  -5574.18
243      20   16      52  1879.21  -5574.18
195      18   16      52   747.89  -5574.18
241      20   16      44   438.40  -5574.18


### SELL OPTIMIZATION

In [ ]:
def convert_rsi_signal_sell(df, rsi_period = 14, overbought = 70) :
    if 'close' in df.columns:
        df['rsi'] = pta.rsi(df.close, rsi_period)
        df['signal'] = 'normal'
        df.loc[df['rsi'] > overbought, 'signal'] = 'sell'
        return df
    else :
        return "Error RSI"

In [ ]:
#sell order opm
def backtest_rsi_sell(df, rsi_period = 14, overbought = 70, tp_sell = 50):
    
    t0 = time.time()
    #Genarate signals and details
    df = convert_rsi_signal_sell(df, rsi_period, overbought)
    df['cross_down'] = (df['rsi'] < tp_sell ) & (df['rsi'].shift(1) > tp_sell)
    df['position_price'] = df['open'].shift(-1)
    
    t1 = time.time() - t0
    #Record positions history
    status = 0
    current_order = 'empty'
    df['profit'] = 0
    df['drawdown'] = 0

    t2 = time.time() - t1 - t0
    
    for ind in df.index:
        #Time series run time
        if df['signal'][ind] == 'sell':
            if status == 0:
                order_price = df['position_price'][ind]
                status = 1
                current_order = 'sell'

        if status == 1:
            if df['cross_down'][ind]:
                profit = order_price - df.loc[ind, 'position_price']
                df.loc[ind, 'profit'] = profit
                status = 0
                current_order = 'empty'

        #Drawdown
        if current_order == 'sell':
            df.loc[ind, 'drawdown'] =  order_price - df.loc[ind, 'high']
    
    t3 = time.time() - t2 - t1 - t0
    total_profit = df['profit'].sum()
    max_dd = df['drawdown'].min()

    return df, total_profit, max_dd, (t1, t2, t3)


In [ ]:
#for sell
rsi_periods_range = np.arange(10, 22, 2) #10
OVERBOUGHT_range = np.arange(64,84,2) #8
TP_SELL_range = np.arange(60, 36,-4) #6

START_DATE = datetime(2021, 3, 19) #h1 pull after 2021-3-19
END_DATE = datetime(2022, 12, 31)

df = pull_data(SYMBOL, TIMEFRAME, START_DATE, END_DATE)
sell_opm = []

for _rsi_period in rsi_periods_range:
    for ovb in OVERBOUGHT_range:
        for tp_sell in TP_SELL_range:
            df, result, max_dd, t = backtest_rsi_sell(df, _rsi_period, ovb, tp_sell)
            sell_opm.append((_rsi_period, ovb, tp_sell, result, max_dd))
            print(_rsi_period, ovb, tp_sell, result, max_dd)
print(sell_opm)

10 64 60 14760.980000000074 -5861.559999999998
10 64 56 12963.040000000057 -6056.110000000001
10 64 52 10859.570000000038 -7091.510000000002
10 64 48 10325.870000000015 -7091.510000000002
10 64 44 12786.199999999993 -7364.690000000002
10 64 40 23799.92999999999 -7855.699999999997
10 66 60 9530.280000000059 -5119.590000000004
10 66 56 8820.950000000043 -6056.110000000001
10 66 52 6821.29000000001 -6979.9100000000035
10 66 48 9505.449999999988 -6979.9100000000035
10 66 44 3354.29999999999 -7364.690000000002
10 66 40 14758.530000000008 -7420.469999999994
10 68 60 5332.490000000076 -5119.590000000004
10 68 56 2093.430000000064 -6056.110000000001
10 68 52 929.3300000000545 -6565.75
10 68 48 578.7300000000232 -6565.75
10 68 44 -7917.819999999969 -7364.690000000002
10 68 40 4268.800000000027 -7420.469999999994
10 70 60 383.3300000000236 -5119.590000000004
10 70 56 -5272.629999999972 -6056.110000000001
10 70 52 -8090.719999999979 -6565.75
10 70 48 -3997.8600000000006 -6565.75
10 70 44 -8991.60

### Result

In [ ]:
test_sell = pd.DataFrame(sell_opm)
test_sell.columns = 'period', 'ovb', 'tp_sell', 'profit', 'drawdown'
test_sell.sort_values('profit', ascending= False).head(10)

,period,ovb,tp_sell,profit,drawdown
5,10,64,40,23799.93,-7855.70
65,12,64,40,18668.73,-9823.41
190,16,66,44,17115.50,-9213.68
184,16,64,44,15827.96,-9631.85
71,12,66,40,15347.54,-9823.41
0,10,64,60,14760.98,-5861.56
11,10,66,40,14758.53,-7420.47
60,12,64,60,14174.26,-6056.11
1,10,64,56,12963.04,-6056.11
4,10,64,44,12786.20,-7364.69


In [ ]:
test_buy.sort_values('profit', ascending= False).head(10)

,period,ovs,tp_buy,profit,drawdown
258,20,22,40,9488.16,-8187.99
157,16,20,44,8362.18,-8187.99
261,20,22,52,7357.24,-8187.99
103,14,18,44,5824.47,-8187.99
104,14,18,48,5689.76,-8187.99
211,18,22,44,5399.58,-9935.88
156,16,20,40,4866.48,-8187.99
259,20,22,44,4545.76,-8187.99
50,12,16,48,4483.99,-8187.99
260,20,22,48,4189.47,-8187.99
